In [1]:
d = '../data/pre/' # raw data directory

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
aC = pd.read_csv('../data/pre/app_categories.csv')

user_installedapps = pd.read_csv('%suser_installedapps.csv' % d)
ui_cates = pd.merge(user_installedapps, aC, on='appID', how='left')
ui_dum_cats = pd.get_dummies(ui_cates['appCategory'], prefix='appCategory', dummy_na=True)
ui_cates = pd.concat([ui_cates, ui_dum_cats], axis=1)
print('\n\n',ui_cates.head(), len(ui_cates))



    userID  appID  appCategory  appCategory_0.0  appCategory_1.0  \
0       1    357          201                0                0   
1       1    360          201                0                0   
2       1    362          409                0                0   
3       1    365          301                0                0   
4       1    375          203                0                0   

   appCategory_2.0  appCategory_103.0  appCategory_104.0  appCategory_105.0  \
0                0                  0                  0                  0   
1                0                  0                  0                  0   
2                0                  0                  0                  0   
3                0                  0                  0                  0   
4                0                  0                  0                  0   

   appCategory_106.0       ...         appCategory_401.0  appCategory_402.0  \
0                  0       ...          

In [3]:
ui = ui_cates.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'appCategory_' in col]].values, axis=0))).reset_index()

ui.columns = ['userID', 'insAppCates']

print('\nDev info\n',ui.head(), len(ui))


Dev info
    userID                                        insAppCates
0       1  [37, 0, 0, 1, 0, 0, 0, 2, 0, 0, 9, 5, 0, 1, 0,...
1       9  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2      10  [46, 0, 1, 1, 0, 0, 1, 0, 0, 0, 5, 2, 0, 1, 0,...
3      12  [21, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,...
4      14  [21, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 1, 0,... 1446105


In [57]:
ui.insAppCates.head().values

array([ [37, 0, 0, 1, 0, 0, 0, 2, 0, 0, 9, 5, 0, 1, 0, 1, 8, 0, 0, 1, 0, 0, 1, 3, 0, 2, 8, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [46, 0, 1, 1, 0, 0, 1, 0, 0, 0, 5, 2, 0, 1, 0, 0, 7, 0, 0, 3, 3, 4, 0, 4, 1, 0, 18, 0],
       [21, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 1, 2, 0, 0, 4, 1, 1, 0, 0],
       [21, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 1, 0, 0, 5, 0, 3, 0, 1, 0, 0, 2, 0, 2, 19, 0]], dtype=object)

In [5]:
user_app_actions = pd.read_csv('%suser_app_actions.csv' % d)

uact_cate = pd.merge(user_app_actions, aC, on='appID', how='left')
uact_dum_cats = pd.get_dummies(uact_cate['appCategory'], prefix='appActCategory', dummy_na=True)
uact_cate = pd.concat([uact_cate, uact_dum_cats], axis=1)

print('\n\nDev info...', ...)



Dev info... Ellipsis


In [6]:
uact_cate['installTime_d'] = uact_cate.installTime.map(lambda x: int(str(x)[0:2]))
uact_cate = uact_cate.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'appActCategory_' in col]].values, axis=0))).reset_index()
uact_cate.columns = ['userID', 'actApps']

In [7]:
print('\n\n',uact_cate.head(), len(uact_cate))



    userID                                            actApps
0       1  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       5  [2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, ...
2      12  [2, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...
3      14  [4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      19  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ... 781112


In [49]:
te_df = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)
te_df = pd.merge(te_df, ui, on='userID', how='left')
te_df = pd.merge(te_df, uact_cate, on='userID', how='left')
te_df.actApps.fillna(str([0] *28), inplace=True)
te_df.insAppCates.fillna(str([0] *28), inplace=True)
te_df.to_csv('../data/pre/new_with_lists_test.csv', index=False)

In [ ]:
tr_df = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
tr_df = pd.merge(tr_df, ui, on='userID', how='left')
tr_df = pd.merge(tr_df, uact_cate, on='userID', how='left')
tr_df.actApps.fillna(str([0] *28), inplace=True)
tr_df.insAppCates.fillna(str([0] *28), inplace=True)
tr_df.to_csv('../data/pre/new_with_lists_train.csv', index=False)

In [53]:
te = pd.read_csv('../data/pre/new_with_lists_test.csv')

In [55]:
te.insAppCates.head().values

array([ '[15, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 3, 0, 2, 7, 0]',
       '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]',
       '[31, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 1, 0, 6, 1, 0, 7, 0, 0, 2, 0, 0, 2, 7, 0, 3, 8, 0]',
       '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]',
       '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'], dtype=object)

In [58]:
tr_df = pd.read_csv('../data/pre/new_with_lists_train.csv')
te_df_ = pd.read_csv('../data/pre/new_with_lists_test.csv')
va_df = tr_df.loc[tr_df['clickTime_d'] == 24]

In [59]:
tr_ui_ = pd.DataFrame([eval(r) for r in tr_df.insAppCates.values])
te_ui_ = pd.DataFrame([eval(r) for r in te_df_.insAppCates.values])
va_ui_ = pd.DataFrame([eval(r) for r in va_df.insAppCates.values])

In [60]:
tr_ua_ = pd.DataFrame([eval(r) for r in tr_df.actApps.values])
te_ua_ = pd.DataFrame([eval(r) for r in te_df_.actApps.values])
va_ua_ = pd.DataFrame([eval(r) for r in va_df.actApps.values])

In [61]:
tr_ui_.to_csv('../data/pre/new_tr_ui.csv', header=None, index=None)
te_ui_.to_csv('../data/pre/new_te_ui.csv', header=None, index=None)
va_ui_.to_csv('../data/pre/new_va_ui.csv', header=None, index=None)

tr_ua_.to_csv('../data/pre/new_tr_ua.csv', header=None, index=None)
te_ua_.to_csv('../data/pre/new_te_ua.csv', header=None, index=None)
va_ua_.to_csv('../data/pre/new_va_ua.csv', header=None, index=None)

In [66]:
len(te_ua_)

338489

In [78]:
te_ui = pd.read_csv('../data/pre/new_tr_ui.csv', header=None)

In [80]:
len(te_ui)

3749528

In [81]:
te_ui

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,15,0,1,1,0,0,0,1,0,0,...,0,2,0,0,0,2,1,2,5,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23,0,1,1,0,1,1,8,0,0,...,0,0,0,0,0,3,1,1,17,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,9,0,0,0,0,0,0,0,0,0,...,0,1,0,3,0,3,0,0,7,0
7,39,0,0,1,0,0,0,0,0,0,...,4,1,1,0,1,4,1,0,19,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,29,0,0,1,2,1,6,0,0,0,...,5,1,0,3,2,2,4,2,14,0
